In [17]:
import json
import numpy as np
import pandas as pd

In [2]:
df_node = pd.read_csv('data/nodes.csv')
df_edge = pd.read_csv('data/edges.csv')

In [3]:
df_edge['source'] = df_edge['source'].astype(str)
df_edge['target'] = df_edge['target'].astype(str)
df_node['id'] = df_node['id'].astype(str)
print(df_node.shape)
print(df_edge.shape)

(187, 4)
(33854, 5)


In [8]:
## Select edges from df_edge
select_edges = []
for index,row in df_node.iloc[:].iterrows():
    name = row['name']
    df_row = df_edge.loc[df_edge['src'] == name]

    ### Only select closest 40 edges with weight >= 0.15
    myrow = df_row[df_row['weight']>= 0.15].head(40)
    # print(myrow.shape)
    if myrow.shape[0] < 1:
        ### For nodes weight less than 0.15 edges, select closest 10 edges with weight >= 0.05
        myrow = df_row[df_row['weight']>= 0.05].head(10)
        # print(myrow.shape)
        # print(myrow)
    select_edges.append(myrow)

In [11]:
## Create final dataframe for selected edges
df_se = pd.concat(select_edges)
df_e = df_se[df_se['weight']>=0.05]
df_e['weight'] = round(df_e['weight'],2)
df_e

,source,target,weight,src,dst
0,0,1,0.71,老实憨厚的子衿,老实憨厚的笑笑
1,2,3,0.69,贝拉kira,嘉然今天吃什么
5,2,6,0.61,贝拉kira,向晚大魔王
6,2,7,0.60,贝拉kira,乃琳Queen
2,4,5,0.66,薇Steria,内德维德
...,...,...,...,...,...
4327,186,172,0.06,你的影月月,雷雨Lyy
4721,171,52,0.05,菠萝赛东°,Asaki大人
4822,171,19,0.05,菠萝赛东°,阿梓从小就很可爱
4943,178,19,0.05,不知原深,阿梓从小就很可爱


In [12]:
## Create final dataframe for selected node
df_n = df_node[df_node['id'].isin(list(df_e.source))]
df_n['category'] = df_n['category'] - 1

### Normalize value
# df_n['value'] = np.log2(df_n.value)
df_n['symbolSize'] = round((df_n.value-df_n.value.min())/(df_n.value.max()-df_n.value.min())*70 + 10, 2)
df_n = df_n.sort_values(['category','value'],ascending=[True,False])

### Remove singleton category
non_singleton = df_n.category.value_counts()[df_n.category.value_counts()>1].index.to_list()
df_n = df_n[df_n['category'].isin(non_singleton)]
df_n

<ipython-input-12-48fece862653>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_n['category'] = df_n['category'] - 1
<ipython-input-12-48fece862653>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_n['symbolSize'] = round((df_n.value-df_n.value.min())/(df_n.value.max()-df_n.value.min())*70 + 10, 2)


,id,name,value,category,symbolSize
17,11,咩栗,11803,0,28.24
6,10,呜米,10064,0,25.00
127,74,扇宝,7555,0,20.32
101,85,早稻叽,7462,0,20.15
66,91,白神遥Haruka,7110,0,19.50
...,...,...,...,...,...
116,137,黑桐谷歌,4170,22,14.02
148,159,猫神辣椒酱,5040,23,15.64
147,158,守护茶茶,5039,23,15.63
178,175,小霖QL,13680,24,31.74


In [14]:
set_category = set(list(df_n.category))
print(f"Total {len(set_category)} lencategory we have: {set_category}")

Total 25 lencategory we have: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24}


## Export to text file

- For js/javascript in HTML

In [15]:
result_node = df_n.to_dict(orient='records')
result_edge = df_e[['source','target','weight']].to_dict(orient='records')

In [20]:
with open('node.json', 'w') as file:
     file.write(json.dumps(result_node,ensure_ascii=False))

with open('edges.json', 'w') as file:
     file.write(json.dumps(result_edge,ensure_ascii=False))